<a href="https://colab.research.google.com/github/Chalkina/sql_homework/blob/main/DS_Homework_1_Python_15122022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests 
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd
from collections import Counter

In [3]:
offset_ = 1
limit_ = 100
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
objects_data.get('data').get('list')[0]

{'hobjId': 39024,
 'objId': 13383,
 'developer': {'devId': 306,
  'shortName': '2МЕН ГРУПП',
  'fullName': '2МЕН ГРУПП',
  'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
  'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
  'companyGroup': 5776,
  'devInn': '7701651356'},
 'rpdRegionCd': 72,
 'objAddr': 'г Тюмень',
 'objFloorMin': 4,
 'objFloorMax': 20,
 'objElemLivingCnt': 68,
 'objReady100PercDt': '2022-12-31',
 'objSquareLiving': 6296.55,
 'objCompensFundFlg': 1,
 'objProblemFlg': 0,
 'latitude': 57.1519,
 'longitude': 65.5536,
 'problemFlag': 'NONE',
 'siteStatus': 'Строящиеся',
 'objGuarantyEscrowFlg': 0,
 'residentialBuildings': 1,
 'rpdNum': '72-000095',
 'pdId': 28105,
 'buildType': 'Жилое',
 'objStatus': 0}

In [4]:
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
objids[:10]

[13383, 13398, 13397, 13399, 36912, 10891, 8809, 8810, 39176, 39175]

In [6]:
d = []
for obj in objids:
  url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj}'
  res = requests.get(url)
  d.append(res.json())

In [26]:
# Создание DataFrame из json
df = pd.json_normalize(d)

In [27]:
df[:10]

,errcode,data.id,data.hobjId,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,...,data.metro.colors,data.transportDistIndex,data.transportDistIndexValue,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.airQualityIndexValue,data.complexShortNm,data.greenAreaIndexValue
0,0,13383,39024,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,13398,37165,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,13397,37201,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,13399,34754,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,36912,117248,337586,10296,CЗ СТРОЙРЕСУРС,CЗ СТРОЙРЕСУРС,CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК СТРОЙРЕСУРС,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,10891,6541,23054,3006,ААК ВОЛОГДААГРОСТРОЙ,ААК ВОЛОГДААГРОСТРОЙ,АКЦИОНЕРНАЯ АГРОСТРОИТЕЛЬНАЯ КОМПАНИЯВОЛОГДААГ...,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,8809,4071,18074,1955,АБСОЛЮТ,АБСОЛЮТ,АБСОЛЮТ,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,8810,39960,18074,1955,АБСОЛЮТ,АБСОЛЮТ,АБСОЛЮТ,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,39176,119604,435352,1146,АВА ДЕВЕЛОПМЕНТ ПЛЮС,АВА ДЕВЕЛОПМЕНТ ПЛЮС,АВА ДЕВЕЛОПМЕНТ ПЛЮС,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,39175,119603,435351,1146,АВА ДЕВЕЛОПМЕНТ ПЛЮС,АВА ДЕВЕЛОПМЕНТ ПЛЮС,АВА ДЕВЕЛОПМЕНТ ПЛЮС,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Cохранить pandas dataframe в excel
df_excel = df.to_excel("output.xlsx")

In [14]:
# Cохранить pandas dataframe в pickle
df_pickle = pd.DataFrame(df, columns=['data.list', 'data.total']).to_pickle('home_frq.pkl')

In [25]:
# Сохранение DataFrame в БД
import sqlite3
conn = sqlite3.connect('houses_database')
df_sql = df.to_sql('houses', conn, if_exists='replace', index = False)